In [1]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interactive
import plotly as plt
import plotly.express as px
import plotly.graph_objects as go
import io
from IPython.display import display, clear_output
from traitlets import traitlets
import methods as tm
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
header = widgets.HTML(
    value='<h1 align="center">TrendPy Webapp<h1>\
    <h3 align="center">Visualization of trends in (time series) data<h3>')
header

HTML(value='<h1 align="center">TrendPy Webapp<h1>    <h3 align="center">Visualization of trends in (time serie…

In [3]:
header2 = widgets.HTML(
    value='<h3 align="center">Upload a .csv file here<h3>')
header2

HTML(value='<h3 align="center">Upload a .csv file here<h3>')

In [4]:
box_layout = widgets.Layout(display='flex',
                justify_content='center')
uploaded_excel_file = widgets.FileUpload(accept=".csv", multiple=False)
container = widgets.HBox([uploaded_excel_file],layout=box_layout)

display(container)

In [5]:
#create pandas dataframe
out = widgets.Output()
def create_pandas_dataframe(b):
    
    # resetting everything for a new calculation
    with out:
        clear_output()
    with out2:
        clear_output()
    fig_widget.update_traces(x=[],y=[],selector=({'name':['trendline','datapoints']}))
    trend_dropdown.value='No trendline'
    polynomial_deg_selection.disabled=True
    r2_checkbox.value=False
    r2_checkbox.disabled=True
        
    #checking whether data has been uploaded succesfully
    if (uploaded_excel_file.value=={}):
        button.button_style='warning'
        with out:
            print("No data entered. Please upload a .csv file.")
    
    else:
        try:
            
            
            button.button_style='success'
        
            #transforming the uploaded .csv file back to the type .csv in order to then create a pandas dataframe
            time_series_file = list(uploaded_excel_file.value.values())[0]
            content=time_series_file['content']
            content=io.StringIO(content.decode('utf-8'))
            time_series_data = pd.read_csv(content)
        
            #filling the drop down menus with the columns of the dataframe as options
            input_dropdown.options = time_series_data.select_dtypes(include='number').columns
            output_dropdown.options = time_series_data.select_dtypes(include='number').columns
            b.value = time_series_data
            
            with out: 
                print("Data entered. Please select the X and Y values now. (Only works if header is in the first row of your file.)")
        except:
            button.button_style='warning'
            with out:
                print('Invalid input! Make sure that header of the file is in first row and general format is correct!')
            

In [6]:
#using a loaded button in order to make the pandas dataframe of the function above available outside of the function
class LoadedButton(widgets.Button):

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        self.add_traits(value=traitlets.Any(value))

button = LoadedButton(description='Start calculation', disable=False, tooltip='Click to start calculation', icon='check', button_style='')
button.on_click(create_pandas_dataframe)


container2 = widgets.HBox([button],layout=box_layout)
container_out = widgets.HBox([out],layout=box_layout)
display(container2,container_out)


In [7]:
header4 = widgets.HTML(
    value='<p align="center" style="font-size:16px">Elements down below are interactive, button above only has to be pressed\
            if a new excel-file has been uploaded<p>')
header4

HTML(value='<p align="center" style="font-size:16px">Elements down below are interactive, button above only ha…

In [12]:
# creates the graphics based on the input parameters defined in the interactive widgets below
def create_graphics(values_in,values_out,trend,deg=2,r2=False):
    if trend == 'polynomial':
        polynomial_deg_selection.disabled=False
    else:
        polynomial_deg_selection.disabled=True
        
    if values_in == '' or values_out=='':
        print("Please select your X and Y values in the dropdown menus above.")
    else:
        try: # sometimes an error is occuring when user hasn't changed the standard dropdown X and Y values yet
             # through try/except the user does not see the error and won't notice it, because standard values are
             # usually not going to be used
            
            # updateing the plot based on the input data
            fig_widget.layout.xaxis.title = values_in
            fig_widget.layout.yaxis.title = values_out
            fig_widget.update_traces(x=button.value[values_in], y=button.value[values_out], selector=({'name':'datapoints'}))
            if trend!="No trendline":
                with out2:
                    clear_output()
                r2_checkbox.disabled=False
                calculate_trendline(values_in, values_out, trend, deg, r2) # function that calculates and draws trendline is called
            else:
                fig_widget.update_traces(x=[],y=[],selector=({'name':'trendline'}))
                r2_checkbox.disabled=True
        except:
            pass


In [9]:
# defining the interactive dropdown options
input_dropdown = widgets.Dropdown(
    options=[''],
    value='',
    description='Input (X):',
    disabled=False,
    )

output_dropdown = widgets.Dropdown(
    options=[''],
    value='',
    description='Target (Y):',
    disabled=False,
    )

trend_dropdown = widgets.Dropdown(
    options=['No trendline','linear','polynomial','trigonometric'],
    value='No trendline',
    description='trendline:',
    disabled=False,
    )

polynomial_deg_selection = widgets.Dropdown(
    options=[2,3,4,5,6],
    value=2,
    description='deg (polyn.)',
    disabled=True,
    )

r2_checkbox = widgets.Checkbox(
    value=False,
    description='Show R2 score',
    disabled=False,
    indent=True
)

# arrangement of dropdown menus
dropdown_elements = widgets.HBox([input_dropdown, output_dropdown], layout=box_layout)
trend_selection = widgets.HBox([trend_dropdown, polynomial_deg_selection], layout=box_layout)
r2_container = widgets.HBox([r2_checkbox],layout=box_layout)


# making all the widgets interactive, so no button needs to be pressed when changing a dropdown value      
out2 = widgets.interactive_output(create_graphics, {'values_in': input_dropdown, 'values_out': output_dropdown, \
                                                    'trend':trend_dropdown, 'deg':polynomial_deg_selection, \
                                                    'r2':r2_checkbox})

out2_container = widgets.HBox([out2],layout=box_layout)

display(dropdown_elements, trend_selection, r2_container, out2_container)


In [10]:
# defining a figure widget, that is created in the beginning and remains static, only graphs are updated
# it is needed to define all needed graphs as empty ones here, so that you can use update_traces
fig_widget = go.FigureWidget()
fig_widget.layout.margin=dict(l=120, r=120, b=25, t=25)
fig_widget.layout.height=400
fig_widget.add_scatter(mode='markers', name='datapoints')
fig_widget.add_scatter(mode='lines', name='trendline', line=dict(shape='spline'))
fig_widget

FigureWidget({
    'data': [{'mode': 'markers', 'name': 'datapoints', 'type': 'scatter', 'uid': '955adbaf-876f…

In [11]:
# calculating and drawing the trendline based on selected options
def calculate_trendline(values_in, values_out, trend, deg, r2):
    sorted_df = button.value.sort_values(by=values_in) # without sorting trendlines can not be plotted correctly
    x_values=sorted_df[values_in].to_numpy()
    y_values=sorted_df[values_out].to_numpy()
    
    # in all cases coefficients are calculated first by calling function from methods.py
    # after that the predicted y-values are calculated by calling function from methods.py
    # with the data from y_pred-function it is then possible to plot the trendline
    # at last r2-score is calculated
    if trend=='linear':
        coefs=tm.linReg(x_values,y_values)
        print('coefficients: ', coefs)
        y_pred=tm.pred('linReg',coefs, x_values)
        fig_widget.update_traces(x=sorted_df[values_in], y=y_pred, selector=({'name':'trendline'}));
        if r2==True:
            r2_score = tm.r2(y_values, y_pred)
            print('R2-score:     ', r2_score)
        
    elif trend=='polynomial':
        try:
            coefs = tm.polReg(x_values,y_values,deg)
            print('coefficients: ', coefs)
            y_pred = tm.pred('polReg', coefs, x_values)
            fig_widget.update_traces(x=sorted_df[values_in], y=y_pred, selector=({'name':'trendline'}), line=dict(shape='spline'));
            if r2==True:
                r2_score = tm.r2(y_values, y_pred)
                print('R2-score:     ', r2_score)
        except:
            print('Selected regression might not be a good fit for the entered values! Please lower degree or choose other regression!')
    elif trend=='trigonometric':
        try:
            coefs = tm.trigReg(x_values,y_values)
            print('coefficients: ', coefs)
            y_pred = tm.pred('trigReg', coefs, x_values)
            fig_widget.update_traces(x=sorted_df[values_in], y=y_pred, selector=({'name':'trendline'}));
            if r2==True:
                r2_score = tm.r2(y_values, y_pred)
                print('R2-score:     ', r2_score)
        except:
            print('Selected regression might not be a good fit for the entered values! Please choose other regression!')